In [ ]:
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML
font = {'size'   : 15}
matplotlib.rc('font', **font)

fft = np.fft.fft
ifft = np.fft.ifft

$\newcommand{\fluctint}[1]{\llbracket #1 \rrbracket}$
In this notebook we solve the variable-coefficient acoustics equations

\begin{align}
p_t + K(x) u_x & = 0 \\
\rho(x) u_t + p_x & = 0
\end{align}

where $p(x,t), u(x,t)$ are the pressure (relative to a background state) and velocity, respectively.  We consider a medium in which $K(x), \rho(x)$ are periodic with period $\delta=1$.

We also solve the homogenized equations (from eqn. (5.17) of LeVeque & Yong, with $G(\sigma)=1$ and $\sigma=-p$)

\begin{align}
    \overline{K^{-1}} \overline{p}_t + \overline{u}_x & = \delta C_1 \overline{u}_{xx} + \delta^2 C_{12} u_{xxx} \\
    \overline{\rho} \  \overline{u}_t + \overline{p}_x & = -\delta C_1 \overline{p}_{xx} + \delta^2 C_{22} p_{xxx}
\end{align}

where $\overline{f}=\int_0^1 f(x)dx$ and

$$
    C_1 = \overline{K^{-1}[\![\rho]\!]}
$$

with

$$
    [\![f]\!](y) = \int_0^y\left\{f(\xi)\right\}d\xi  - \int_0^1 \int_0^\tau \left\{f(\xi)\right\}d\xi d\tau.
$$
where $\left\{f(y)\right\} = f(y) - \int_0^1 f(x)dx$.


$$
v = \begin{bmatrix} p \\ u \end{bmatrix}
$$

# Solution of the variable-coefficient system

Here we use a pseudospectral method in space and 3rd-order Runge-Kutta in time.

In [ ]:
def rk3(v,xi,rhs,dt):
    y2 = v + dt*rhs(v,xi)
    y3 = 0.75*v + 0.25*(y2 + dt*rhs(y2,xi))
    v_new = 1./3 * v + 2./3 * (y3 + dt*rhs(y3,xi))
    return v_new


    
def solve_wave_equation(m,dt):
    L = 100
    x = np.arange(-m/2,m/2)*(L/m)
    xi = np.fft.fftfreq(m)*m*2*np.pi/L

    p0 = np.exp(-0.36*(x+2)**2)
    u0 = np.zeros_like(p0)
    v = np.zeros((2,len(x)))
    v[0,:] = p0; v[1,:] = u0
    tmax = 168.0

    K = 4 + np.sin(2*np.pi*x)
    rho = 4 + np.sin(2*np.pi*x)
    
    def rhs(v, xi):
        d = np.zeros_like(v)
        p = v[0,:]
        u = v[1,:]
        d[0,:] = - K * np.real(ifft(1j*xi*fft(u)))
        d[1,:] = -1./rho * np.real(ifft(1j*xi*fft(p)))
        return d

    num_plots = 50
    nplt = np.floor((tmax/num_plots)/dt)
    nmax = int(round(tmax/dt))

    frames = [v.copy()]
    tt = [0]
    
    for n in range(1,nmax+1):
        v_new = rk3(v,xi,rhs,dt)

        v = v_new.copy()
        t = n*dt
        # Plotting
        if np.mod(n,nplt) == 0:
            frames.append(v.copy())
            tt.append(t)
    return frames, x, tt, xi
    


def plot_solution(frames, x, tt, xi):
    fig = plt.figure(figsize=(12,8))
    axes = fig.add_subplot(111)
    line, = axes.plot(x,frames[0][0,:],lw=3)
    xi_max = np.max(np.abs(xi))
    axes.set_xlabel(r'$x$',fontsize=30)
    plt.close()

    def plot_frame(i):
        line.set_data(x,frames[i][0,:])
        axes.set_title('t= %.2e' % tt[i])
        axes.set_xlim((x[0],x[-1]))
        axes.set_ylim((-0.1,1.1))

    anim = matplotlib.animation.FuncAnimation(fig, plot_frame,
                                              frames=len(frames), interval=100,
                                              repeat=False)
    return HTML(anim.to_jshtml())

In [ ]:
m = 1024
dt = 1.73/(m/2)
frames, x, tt, xi = solve_wave_equation(m,dt)

In [ ]:
plot_solution(frames, x, tt, xi)

The solution shows the presence of significant effective dispersion, as we expect based on the homogenized equations.

Now we compute the constants $C_1, C_{12}, C_{21}$ (see LeVeque & Yong eqn. (5.18) for the formulas).

In [ ]:
from scipy.integrate import quad

def bracket(f):
    # Compute the double bracket of a function
    mean = quad(f,0,1)[0]
    brace = lambda y: f(y)-mean
    brack_nzm = lambda y: quad(brace,0,y)[0]
    mean_bracket = quad(brack_nzm,0,1)[0]
    def brack(y):
        return brack_nzm(y) - mean_bracket
    return brack

rhof = lambda y: 4 + 1.*np.sin(2*np.pi*y)
Kif = lambda y: 1/(4+ 1.*np.sin(2*np.pi*y))
brho = bracket(rhof)
kibr = lambda y: brho(y)*Kif(y)
C1 = quad(kibr,0,1)[0]
C1

We note that $C_1$ vanishes for many simple periodic functions (including sine, cosine, and piecewise-constant).  Even in cases for which it doesn't vanish, it seems to be so small that the next-order dispersive terms dominate.

In [ ]:
bKi = bracket(Kif)
brKir = lambda y: rhof(y)*bKi(y)
C2 = quad(brKir,0,1)[0]
C2

In [ ]:
Kim = quad(Kif,0,1)[0]
rhom = quad(rhof,0,1)[0]
c_hatsq = 1./(Kim*rhom)
c_hatsq

In [ ]:
C12a = lambda y: bKi(y)*brho(y)
C12b = lambda y: brho(y)**2*Kif(y)
C12 = quad(C12a,0,1)[0]*c_hatsq-quad(C12b,0,1)[0]*c_hatsq/rhom
C12

In [ ]:
C22a = C12a
C22b = lambda y: bKi(y)**2*rhof(y)
C22 = quad(C22a,0,1)[0]*c_hatsq-quad(C22b,0,1)[0]*c_hatsq/Kim
C22

Now we solve the homogenized equations in Fourier space.  Unfortunately, the dispersion relation of the truncated homogenized equations when we keep up to $O(\delta^2)$ terms has the feature that, for large enough wavenumbers, the frequency becomes complex and this means there are exponentially-growing modes.  For this reason, we cannot go to extremely long times in the solution below.

In [ ]:
L = 100
x = np.arange(-m/2,m/2)*(L/m)
xi = np.fft.fftfreq(m)*m*2*np.pi/L

p0 = np.exp(-0.36*(x+2)**2)
u0 = np.zeros_like(p0)
    
# Exact solution
alpha = C1
delta = 1.
a = -xi/Kim*(1j+delta*alpha*c_hatsq*xi) - 1j*delta**2*C12*xi**3/Kim + 1.e-16
b = -xi/rhom*(1j-delta*alpha*c_hatsq*xi) - 1j*delta**2*C22*xi**3/rhom + 1.e-16
p0hat = fft(p0)
u0hat = fft(u0)
t = 168
phat = np.cosh(t*np.sqrt(a*b))*p0hat + np.sqrt(a/b)*np.sinh(t*np.sqrt(a*b)*u0hat)
p = np.real(ifft(phat))
plt.figure(figsize=(12,12))
plt.plot(x,p)
plt.plot(x,frames[-1][0,:]);
plt.legend(['Homogenized equations','Variable-coefficient equations'])

We see relatively good agreement, with a couple of caveats:
1. The smaller trailing oscillations are not captured by the homogenized equations.  In order to capture them, we should include more terms.
2.  The homogenized solution has a high-frequency oscillation throughout the domain.  This is due to the exponentially-growing modes that blow up a short time after this.  To avoid this, we should truncate the homogenized equations at an odd power of $delta$ (at least, I think this will avoid the problem).

Below you can see that the solution blows up later.

In [ ]:
L = 100
x = np.arange(-m/2,m/2)*(L/m)
xi = np.fft.fftfreq(m)*m*2*np.pi/L

p0 = np.exp(-0.36*(x+2)**2)
u0 = np.zeros_like(p0)
    
# Exact solution
alpha = C1
delta = 1.
a = -xi/Kim*(1j+delta*alpha*c_hatsq*xi) - 1j*delta**2*C12*xi**3/Kim + 1.e-16
b = -xi/rhom*(1j-delta*alpha*c_hatsq*xi) - 1j*delta**2*C22*xi**3/rhom + 1.e-16
p0hat = fft(p0)
u0hat = fft(u0)
t = 300
phat = np.cosh(t*np.sqrt(a*b))*p0hat + np.sqrt(a/b)*np.sinh(t*np.sqrt(a*b)*u0hat)
p = np.real(ifft(phat))
plt.figure(figsize=(12,12))
plt.plot(x,p)